# Faithful Abel–Jacobi Lookup Tables (CUDA / MPS)

GPU-accelerated, cut-aware, sheet-aware Abel–Jacobi lookup tables.
Produces I0, I1, and bridge vector B for arbitrary genus (default g=30).

In [ ]:
# Setup
!pip install -q torch numpy tqdm

import os, time, numpy as np, torch
from tqdm import tqdm
from google.colab import drive

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")

DTYPE = torch.complex64
REAL_DTYPE = torch.float32

print("Device:", DEVICE)

DRIVE_FOLDER = "AJ_Tables_g30"
drive.mount('/content/drive', force_remount=True)
SAVE_DIR = f"/content/drive/MyDrive/{DRIVE_FOLDER}"
os.makedirs(SAVE_DIR, exist_ok=True)

OMEGAS_PATH    = os.path.join(SAVE_DIR, "aj_omegas_genus30.pt")
INTEGRALS_PATH = os.path.join(SAVE_DIR, "aj_integrals_genus30.pt")

In [ ]:
# Geometry
genus = 30
H = W = 96
r_min, r_max = -6.0, 6.0
i_min, i_max = -6.0, 6.0

grid_r = np.linspace(r_min, r_max, W)
grid_i = np.linspace(i_min, i_max, H)

base_point = complex(r_min - 2.0, i_min - 2.0)

def atomic_torch_save(obj, path):
    tmp = path + ".tmp"
    torch.save(obj, tmp)
    os.replace(tmp, path)

def make_hyperelliptic_cuts(g, radius=4.0, jitter=0.25, seed=123):
    rng = np.random.RandomState(seed)
    m = 2*g + 2
    thetas = np.linspace(0, 2*np.pi, m, endpoint=False)
    rng.shuffle(thetas)
    radii = radius * (1.0 + jitter * (rng.rand(m) - 0.5))
    pts = radii * np.exp(1j * thetas)

    remaining = list(range(m))
    cuts = []
    while remaining:
        i = remaining.pop(0)
        pi = pts[i]
        j = min(remaining, key=lambda k: abs(pi-pts[k]))
        remaining.remove(j)
        a, b = pts[i], pts[j]
        mid = 0.5*(a+b)
        a = a + 0.05*(a-mid)
        b = b + 0.05*(b-mid)
        cuts.append((complex(a), complex(b)))
    return cuts[:g+1]

branch_cuts = make_hyperelliptic_cuts(genus)
branch_pts  = [a for ab in branch_cuts for a in ab]

print("Cuts:", len(branch_cuts))

In [ ]:
# Vectorized omega
branch_pts_tensor = torch.tensor(branch_pts, device=DEVICE, dtype=DTYPE).view(1,1,-1)

def omega_torch(k, t):
    diffs = t.unsqueeze(-1) - branch_pts_tensor
    log_diffs = torch.log(diffs)
    log_denom = 0.5 * torch.sum(log_diffs, dim=-1)
    log_num   = k * torch.log(t)
    return torch.exp(log_num - log_denom)

In [ ]:
# Cut intersection helpers
cuts_a = torch.tensor([[a.real, a.imag] for a,b in branch_cuts], device=DEVICE, dtype=REAL_DTYPE)
cuts_b = torch.tensor([[b.real, b.imag] for a,b in branch_cuts], device=DEVICE, dtype=REAL_DTYPE)

def cross2(u, v):
    return u[...,0]*v[...,1] - u[...,1]*v[...,0]

def segment_intersections(base_xy, z_xy, a_xy, b_xy, eps=1e-9):
    p = base_xy.view(1,1,2)
    r = (z_xy - base_xy).view(-1,1,2)
    q = a_xy.view(1,-1,2)
    s = (b_xy - a_xy).view(1,-1,2)

    rxs = cross2(r, s)
    qmp = q - p
    qmpxs = cross2(qmp, s)
    qmpxr = cross2(qmp, r)

    valid = torch.abs(rxs) > eps
    t = qmpxs / (rxs + (~valid)*1.0)
    u = qmpxr / (rxs + (~valid)*1.0)

    hit = valid & (t > 0) & (t <= 1) & (u >= 0) & (u <= 1)
    return torch.where(hit, t, torch.full_like(t, float('inf')))

In [ ]:
# Faithful row integration
def integrate_row_faithful(k, z_row, base_point, steps=300):
    W = z_row.shape[0]
    base_xy = torch.tensor([base_point.real, base_point.imag], device=DEVICE, dtype=REAL_DTYPE)

    z_xy = torch.stack([z_row.real, z_row.imag], dim=1)
    t_cut = segment_intersections(base_xy, z_xy, cuts_a, cuts_b)
    parity = torch.isfinite(t_cut).sum(dim=1) % 2

    t = torch.linspace(0, 1, steps, device=DEVICE, dtype=REAL_DTYPE).view(-1,1)
    diff = z_row - base_point
    path = base_point + t.type(DTYPE) * diff.view(1,-1)

    t_exp = t.view(-1,1,1)
    sgn = (torch.isfinite(t_cut).unsqueeze(0) & (t_exp >= t_cut.unsqueeze(0))).sum(dim=2) % 2
    sgn = 1.0 - 2.0*sgn.float()

    vals = omega_torch(k, path) * sgn
    integral = torch.sum(vals, dim=0) * (diff / steps)
    return integral, parity

In [ ]:
# Build tables I0, I1, B
COMMON_META = {
    "genus": genus,
    "branch_cuts": branch_cuts,
    "branch_pts": np.array(branch_pts),
    "grid_r": grid_r,
    "grid_i": grid_i,
    "meta": {"base_point": base_point, "grid_shape": (H,W)}
}

I0 = torch.zeros(genus, H, W, dtype=DTYPE)
I1 = torch.zeros_like(I0)

# Bridge vector B from midpoint of first cut
mid = 0.5*(branch_cuts[0][0] + branch_cuts[0][1])
mid_t = torch.tensor([mid], device=DEVICE, dtype=DTYPE)
B = torch.zeros(genus, dtype=DTYPE)

for k in range(genus):
    val, _ = integrate_row_faithful(k, mid_t, base_point)
    B[k] = 2.0 * val[0]

grid_r_t = torch.tensor(grid_r, device=DEVICE, dtype=REAL_DTYPE)

for k in range(genus):
    for iy in tqdm(range(H), desc=f"I[k={k}]"):
        y = grid_i[iy]
        z_row = torch.complex(grid_r_t, torch.full_like(grid_r_t, y))
        vals, parity = integrate_row_faithful(k, z_row, base_point)
        vals = vals.cpu()
        parity = parity.cpu()
        for ix in range(W):
            if parity[ix] == 0:
                I0[k,iy,ix] = vals[ix]
                I1[k,iy,ix] = B[k] - vals[ix]
            else:
                I1[k,iy,ix] = vals[ix]
                I0[k,iy,ix] = B[k] - vals[ix]

payload = {**COMMON_META, "I0": I0, "I1": I1, "B": B}
atomic_torch_save(payload, INTEGRALS_PATH)

print("Saved faithful integrals.")